### Creation of Dataset

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Number of rows
num_rows = 4000

years = np.random.choice([2020, 2021, 2022, 2023], num_rows, p=[0.25, 0.25, 0.25, 0.25])
branches = np.random.choice(['Branch_A', 'Branch_B', 'Branch_C', 'Branch_D'], num_rows)
regions = np.random.choice(['North', 'South', 'East', 'West'], num_rows)

# Create data
data = {
    'Year': years,
    'Branch': branches,
    'Region': regions,
    'Transaction_Times': np.random.normal(loc=5, scale=1, size=num_rows).round(2),  # in minutes
    'Cost_Per_Transaction': np.random.normal(loc=1, scale=0.2, size=num_rows).round(2),  # in dollars
    'Error_Rates': np.random.uniform(0, 0.05, num_rows).round(4),  # as a percentage
    'Employee_Productivity': np.random.normal(loc=20, scale=5, size=num_rows).round(2),  # in transactions/hour
    'Customer_Satisfaction': np.random.uniform(1, 5, num_rows).round(1),  # rating scale 1 to 5
    'System_Uptime': np.random.uniform(99.0, 100.0, num_rows).round(2),  # as a percentage
    'Operational_Cost_Savings': np.random.normal(loc=10000, scale=2000, size=num_rows).round(2),  # in dollars
    'Loan_Processing_Times': np.random.normal(loc=20, scale=5, size=num_rows).round(2),  # in days
    'Fraud_Detection_Rate': np.random.uniform(0.1, 1, num_rows).round(2),  # as a percentage
    'Account_Openings': np.random.randint(10, 100, num_rows),  # count
    'Account_Closures': np.random.randint(1, 50, num_rows),  # count
    'ATM_Uptime': np.random.uniform(95.0, 99.9, num_rows).round(2),  # as a percentage
    'Loan_Default_Rate': np.random.uniform(0.01, 0.2, num_rows).round(3),  # as a percentage
    'Customer_Complaints': np.random.randint(0, 10, num_rows),  # count
    'Net_Promoter_Score': np.random.uniform(-100, 100, num_rows).round(1),  # NPS score
    'Employee_Turnover_Rate': np.random.uniform(0, 20, num_rows).round(2)  # as a percentage
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Introduce missing values for columns other than Year, Branch, and Region
columns_with_missing_values = [col for col in df.columns if col not in ['Year', 'Branch', 'Region']]
for col in columns_with_missing_values:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

# Introduce outliers
outliers_indices = np.random.choice(df.index, size=int(num_rows * 0.01), replace=False)
df.loc[outliers_indices, 'Transaction_Times'] *= 3
df.loc[outliers_indices, 'Cost_Per_Transaction'] *= 2
df.loc[outliers_indices, 'Error_Rates'] *= 5

# Recode some values for demonstration
df['Customer_Satisfaction'] = df['Customer_Satisfaction'].apply(lambda x: 5 if x > 4 else x)
df['System_Uptime'] = df['System_Uptime'].apply(lambda x: 99.99 if x > 99.9 else x)

# Additional float values
df['Operational_Efficiency_Index'] = np.random.uniform(0.5, 1.5, num_rows).round(2)

# Sort by Year
df = df.sort_values(by='Year').reset_index(drop=True)




## Clean the Dataset
### 1. Fill the numerical blanks with mean
### 2. Fill the categorical blanks with mode 

In [2]:
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    df[col].fillna(df[col].mean(), inplace=True)



categorical_columns = df.select_dtypes(include=[object]).columns
for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)


#### Rounding off decimal values to 2 decimal places

In [3]:
df[numeric_columns] = df[numeric_columns].round(2)


#### 1. Recode Customer Satisfaction values to integer and range from 1 to 3
#### 2. Handle outliers by capping them to the lower and upper bounds

In [4]:
df['Customer_Satisfaction'] = df['Customer_Satisfaction'].apply(lambda x: np.random.randint(1, 4) if not np.isnan(x) else np.nan).astype('Int64')

for column in numeric_columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])


### Summarize Data

In [5]:
summary = df.describe().transpose()
frequency_tables = {col: df[col].value_counts() for col in df.columns if df[col].dtype == 'object' or df[col].dtype.name == 'category'}

print("summary:\n" , summary)
print("\n")
print("frequency_tables:\n\n", frequency_tables)

summary:
                                count         mean          std         min  \
Year                          4000.0  2021.494000     1.123735  2020.00000   
Transaction_Times             4000.0     5.030593     0.971517     2.55500   
Cost_Per_Transaction          4000.0     1.007812     0.191506     0.52000   
Error_Rates                   4000.0     0.026533     0.015119     0.00000   
Employee_Productivity         4000.0    20.013420     4.669728     8.23500   
System_Uptime                 4000.0    99.502075     0.277333    99.00000   
Operational_Cost_Savings      4000.0  9967.965835  1864.669980  5236.02375   
Loan_Processing_Times         4000.0    19.873775     4.808808     7.62000   
Fraud_Detection_Rate          4000.0     0.545950     0.246438     0.10000   
Account_Openings              4000.0    55.748750    24.569896    10.00000   
Account_Closures              4000.0    25.273000    13.323500     1.00000   
ATM_Uptime                    4000.0    97.463553     

### Save the cleaned and summarized data to a new Excel file

In [6]:
output_file_path = r'C:\Users\Nidhi mishra\OneDrive\Desktop\operational_efficiency_cleaned.xlsx'
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Cleaned_Data', index=False)
    summary.to_excel(writer, sheet_name='Summary')
    for col, freq_table in frequency_tables.items():
        freq_table.to_excel(writer, sheet_name=f'{col}_Frequency')

print(f"Data cleaning and summarization completed. File saved as '{output_file_path}'")

Data cleaning and summarization completed. File saved as 'C:\Users\Nidhi mishra\OneDrive\Desktop\operational_efficiency_cleaned.xlsx'
